# Test queries for a locally installed COVID-19 Knowledge Graph
[Work in progress]

This notebook demonstrates how to run Cypher queries on a locally installed Neo4j database (e.g., for testing). 

**Note:**

This notebook works only on Linux and Mac OS.

This notebook will not work on Binder.

The environmental variable `NEO4J_HOME` must be set to the Neo4j installation directory path.

In [21]:
import os
import time
import pandas as pd
from py2neo import Graph

In [22]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [23]:
NEO4J_HOME = os.getenv('NEO4J_HOME')
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


### Start Neo4j Database

In [24]:
status = !"$NEO4J_HOME"/bin/neo4j status
while not 'Neo4j is running' in status:
    !"$NEO4J_HOME"/bin/neo4j start
    time.sleep(30)
    status = !"$NEO4J_HOME"/bin/neo4j status
    status = str(status)
    print(status)

# sometimes neo4j needs more time to be ready
time.sleep(60)

Directories in use:
  home:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3
  config:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/conf
  logs:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/logs
  plugins:      /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/plugins
  import:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/import
  data:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a2

##### NOTE: Starting Neo4j from this notebook doesn't always work. If the line printed above is `Neo4j is not running`, go back to the top of the Notebook and try starting it again.¶

In [41]:
#graph = Graph("http://localhost:7687/db/covid19sandiego", password="neo4jbinder"
graph = Graph("bolt://132.249.238.185:7687", auth=("reader", "demo"), database='covid19sandiego')

### List Node Metadata

This lists the nodes and their metatdata available in this graph database

In [42]:
query = """
MATCH (n:NodeMetadata)
RETURN n.name, n.shortDescription, n.description, n.example, n.definitionSource, n.dataProviders
"""
graph.run(query).to_data_frame()

,n.name,n.shortDescription,n.description,n.example,n.definitionSource,n.dataProviders
0,Location,Geographic location,A geographic location,"World, ..., Country, State, Country, City, Cru...",None,"[GeoNames, UNSD, USCensus, HUD, JHU]"
1,World,The World,Top level location,None,None,None
2,UNRegion,Continental regions,Continental regions according to the M49 stan...,Americas,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
3,UNSubRegion,Subcontinental regions,Subcontinental regions according to the M49 st...,Latin America and the Caribbean,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
4,UNIntermediateRegion,Subdivisions of subcontinental regions,Subdivisions of subcontinental regions accordi...,Caribbean,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
5,Country,Countries and dependent Territories,Countries and dependent Territories defined b...,United States,http://www.geonames.org/,[GeoNames]
6,Admin1,"State, Province, Municipality","First administrative divisions, e.g, State, Pr...",California,http://www.geonames.org/,"[GeoNames, USCensus]"
7,Admin2,County,Second administrative divisions: County in the US,San Diego County,http://www.geonames.org/,"[GeoNames, USCensus]"
8,City,City,City,San Diego,http://www.geonames.org/,"[GeoNames, USCensus]"
9,PostalCode,Postal Code,"E.g., a ZIP Code is a postal code used by the ...",92121,http://purl.obolibrary.org/obo/OPMI_0000120,"[GeoNames, HUD]"


### List Organisms in KG

In [27]:
query = """
MATCH (p:Organism)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009
3,human,Homo sapiens,taxonomy:9606
4,house mouse,Mus musculus,taxonomy:10090
5,intermediate horseshoe bat,Rhinolophus affinis,taxonomy:59477
6,Malayan horseshoe bat,Rhinolophus malayanus,taxonomy:608659
7,horseshoe bat,Rhinolophus,taxonomy:49442
8,Malayan pangolin,Manis javanica,taxonomy:9974
9,palm civet,Paradoxurus,taxonomy:71116


### List Coronavirus Outbreaks

In [28]:
query = """
MATCH (p:Pathogen)-[:CAUSES]->(o:Outbreak)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy, o.id as outbreak, o.startDate as startDate
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy,outbreak,startDate
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049,COVID-19,2019
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626,MERS,2012
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009,SARS,2003


### List Strains that are mentioned in PubMed Central Articles

In [29]:
query = """
MATCH (p:Publication)-[:MENTIONS]->(s:Strain)<-[:CARRIES]-(h:Host)
RETURN p.id as pmc, s.name as name, s.collectionDate  as collectionDate, h.name as host, h.id as host_id
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame()

,pmc,name,collectionDate,host,host_id
0,pmc:PMC7166309,hCoV-19/pangolin/Guangxi/P4L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
1,pmc:PMC7166309,hCoV-19/pangolin/Guangxi/P5E/2017,2017-01-01,Malayan pangolin,taxonomy:9974
2,pmc:PMC7166309,hCoV-19/pangolin/Guangxi/P5L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
3,pmc:PMC7166309,hCoV-19/pangolin/Guangxi/P2V/2017,2017-01-01,Malayan pangolin,taxonomy:9974
4,pmc:PMC7166309,hCoV-19/pangolin/Guangxi/P1E/2017,2017-01-01,Malayan pangolin,taxonomy:9974
5,pmc:PMC7228214,hCoV-19/pangolin/Guangxi/P4L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
6,pmc:PMC7211627,hCoV-19/pangolin/Guangxi/P5L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
7,pmc:PMC7166309,hCoV-19/pangolin/Guangdong/1/2019,2019-01-01,Malayan pangolin,taxonomy:9974
8,pmc:PMC7228214,hCoV-19/pangolin/Guangdong/1/2019,2019-01-01,Malayan pangolin,taxonomy:9974
9,pmc:PMC7205519,hCoV-19/pangolin/Guangdong/1/2019,2019-01-01,Malayan pangolin,taxonomy:9974


### List Gene and Protein information for Reference Genome
This query lists the genes and proteins encoded by the SARS-CoV-2 reference genome. This is the first genome of SARS-CoV-2 collected in Wuhan on Dec. 5, 2019.

In [30]:
query = """
MATCH (s:Strain)-[:HAS]->(g:Gene)-[:ENCODES]->(p:Protein)
RETURN s.id as referenceGenome, s.name as name, s.collectionDate  as collectionDate, 
       g.name as gene, g.id as geneId, p.name as protein, p.id as protein_id 
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame()

,referenceGenome,name,collectionDate,gene,geneId,protein,protein_id
0,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,ORF1ab,ncbiprotein:NC_045512-266-21555,Replicase polyprotein 1ab,md5:e6608b50fcd6e004708a875615ddf2d9
1,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,ORF1ab,ncbiprotein:NC_045512-266-13483,Replicase polyprotein 1a,md5:e781b58591b8dbdd15f84dcbdec82105
2,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,S,ncbiprotein:NC_045512-21563-25384,Spike glycoprotein,md5:4c35f09aac2f7be4f3cffd30c6aecac8
3,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,ORF3a,ncbiprotein:NC_045512-25393-26220,Protein 3a,md5:f5c8b89ceac3f14e456577557df1ef40
4,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,E,ncbiprotein:NC_045512-26245-26472,Envelope small membrane protein,md5:375e0f905c315e06a99c80b736c125d2
5,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,M,ncbiprotein:NC_045512-26523-27191,Membrane protein,md5:1cd6abff79ad3633e17582eb0e576539
6,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,ORF6,ncbiprotein:NC_045512-27202-27387,Non-structural protein 6,md5:90b50e0be9abd893bd32b163d6933f0c
7,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,ORF7a,ncbiprotein:NC_045512-27394-27759,Protein 7a,md5:f65213344e2e68de1cae4feb9c5e07b1
8,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,ORF7b,ncbiprotein:NC_045512-27756-27887,Protein non-structural 7b,md5:c7f0179da4ca26456ee905081f485cc8
9,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-10,ORF8,ncbiprotein:NC_045512-27894-28259,Non-structural protein 8,md5:3a77ad9207d29beacfbf6f18b0da9e26


### Cases in a specific County (Admin2)

In [33]:
admin2 = 'San Diego County'

query = """
MATCH (c:Cases{date: date("2020-07-09")})-[:REPORTED_IN]->(a:Admin2{name: $admin2})
RETURN a.name as name, c.cummulativeConfirmed as confirmed, c.cummulativeDeaths as deaths
"""
graph.run(query, admin2=admin2).to_data_frame()

,name,confirmed,deaths
0,San Diego County,18402,415


### Aggregate cases by State (Admin1)

In [12]:
query = """
MATCH (o:Outbreak{id: "COVID-19"})<-[:RELATED_TO]-(c:Cases{date: date("2020-04-30")})-[:REPORTED_IN]->(a:Admin2)-[:IN]->(a1:Admin1)
RETURN a1.name as state, sum(c.cummulativeConfirmed) as confirmed, sum(c.cummulativeDeaths) as deaths
ORDER BY deaths DESC
"""
graph.run(query).to_data_frame()

,state,confirmed,deaths
0,New York,304372,23587
1,New Jersey,118174,7228
2,Michigan,39445,3722
3,Massachusetts,61762,3556
4,Pennsylvania,47971,2475
5,Illinois,52065,2324
6,Connecticut,27238,2253
7,California,48631,2004
8,Louisiana,24018,1526
9,Florida,33228,1242


In [14]:
!"$NEO4J_HOME"/bin/neo4j stop

Stopping Neo4j........ stopped


## Old queries need to be updated

### List person demographics and strain information for California

In [14]:
# query = """
# MATCH (a:Admin1)<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
# WHERE a.name = 'California'
# RETURN p.age as age, p.sex as sex, p.exposure_location as exposure_location, s.name as strain, s.clade as clade
# """
# graph.run(query).to_data_frame()

#### Same query using parameterized Cypher
Parameters to Cypher queries can be passed as key-value arguments. Parameters in Cypher are named and are wrapped in curly braces.

In [15]:
# admin1 = 'California'

# query = """
# MATCH (a:Admin1{name: {admin1}})<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
# RETURN p.age as age, p.sex as sex, p.exposure_location as exposure_location, 
#        s.name as strain, s.clade as clade, s.date as date
# ORDER BY s.date
# """
# graph.run(query, admin1=admin1).to_data_frame().head(100)

### Where did clade A originate?

In [16]:
# clade = 'A'

# query = """
# MATCH (s:Strain)--(a:Country)
# WHERE s.clade STARTS WITH {clade}
# RETURN s.clade as clade, s.name, s.date, a.name
# ORDER BY s.date
# """
# graph.run(query, clade=clade).to_data_frame().head(100)

### Find persons that imported the virus from another location

In [17]:
# query = """
# MATCH (c:Admin1)<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
# WHERE c.name <> p.exposure_location
# RETURN c.name as `state/province`, p.age as age, p.sex as sex, p.exposure_location as exposure_location, 
#        s.name as strain, s.clade as clade
# ORDER BY p.exposure_location
# """
# graph.run(query).to_data_frame()